In [67]:
import pandas as pd
import sqlite3
con = sqlite3.connect("database.sqlite")

def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """
    query_output = pd.read_sql_query(query, con)
    return pd.DataFrame(query_output)


In [68]:
# Load the data into a DataFrame
df_country = show_query("SELECT * FROM Country")
df_league = show_query("SELECT * FROM League")
df_match = show_query("SELECT * FROM Match")
df_player = show_query("SELECT * FROM Player")
df_player_attributes = show_query("SELECT * FROM Player_Attributes")
df_team = show_query("SELECT * FROM Team")
df_team_attributes = show_query("SELECT * FROM Team_Attributes")

Maak een functie waarmee je per seizoen voor een gekozen club o.a. het volgende kunt bepalen:

- Aantal wedstrijden in dat seizoen.
- Aantal punten behaald in dat seizoen.
- Gemiddeld aantal punten per wedstrijd.
- Winst-, verlies- en gelijk percentage in uitwedstrijden en thuiswedstrijden.

In [93]:

def details_season(season, club_id):
    """Docstrings
    
    """
    df_home = df_match[(df_match['season'] == season) & (df_match['home_team_api_id'] == club_id)] 
    df_away = df_match[(df_match['season'] == season) & (df_match['away_team_api_id'] == club_id)]
    
    home_points = 0
    for index, rows in df_home.iterrows():
        if rows['home_team_goal'] > rows['away_team_api_id']:
            home_points += 3
        elif rows['home_team_goal'] < rows['away_team_api_id']:
            home_points += 0
        else:
            home_points += 1
    
    away_points = 0
    for index, rows in df_away.iterrows():
        if rows['away_team_goal'] > rows['home_team_api_id']:
            away_points += 3
        elif rows['away_team_api_id'] < rows['home_team_api_id']:
            away_points += 0
        else:
            away_points += 1
    
    amount_of_games = len(df_home) + len(df_away)
    amount_of_points = away_points
        
    print('Amount of games this season: ' + str(amount_of_games))
    print('Amount of points this season: ' + str(amount_of_points))
    
    return details_season
        

In [94]:
details_season('2008/2009', 10228)

Amount of games this season: 34
Amount of points this season: 15


<function __main__.details_season(season, club_id)>

34

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
13587,13588,13274,13274,2009/2010,1,2009-07-31 00:00:00,662550,10228,9803,0,...,7.00,1.40,3.75,7.50,1.42,4.00,7.50,1.44,3.75,7.00
13608,13609,13274,13274,2009/2010,12,2009-10-31 00:00:00,662348,10228,10217,3,...,6.00,1.67,3.40,5.00,1.65,3.50,4.75,1.62,3.50,5.00
13634,13635,13274,13274,2009/2010,15,2009-11-28 00:00:00,662374,10228,10219,3,...,8.50,1.30,4.50,8.00,1.36,4.50,7.00,1.33,4.33,8.50
13653,13654,13274,13274,2009/2010,17,2009-12-12 00:00:00,662393,10228,10235,0,...,2.50,2.38,3.20,2.75,2.50,3.30,2.50,2.50,3.30,2.50
13670,13671,13274,13274,2009/2010,19,2010-01-23 00:00:00,662410,10228,8525,4,...,7.50,1.40,4.00,7.50,1.45,3.75,7.00,1.44,3.75,7.00
13707,13708,13274,13274,2009/2010,22,2010-02-06 00:00:00,662438,10228,9908,2,...,3.20,2.10,3.20,3.25,2.15,3.30,3.00,2.25,3.30,2.88
13715,13716,13274,13274,2009/2010,23,2010-02-13 00:00:00,662446,10228,8593,0,...,1.67,4.50,3.60,1.67,4.50,3.50,1.70,4.33,3.50,1.73
13734,13735,13274,13274,2009/2010,25,2010-02-27 00:00:00,662465,10228,9791,1,...,4.75,1.67,3.60,4.50,1.75,3.40,4.25,1.73,3.50,4.20
13760,13761,13274,13274,2009/2010,28,2010-03-19 00:00:00,662491,10228,8464,4,...,5.50,1.62,3.60,5.00,1.70,3.50,4.50,1.75,3.40,4.20
13777,13778,13274,13274,2009/2010,3,2009-08-15 00:00:00,662265,10228,8277,1,...,4.33,1.67,3.40,5.00,1.70,3.50,4.50,1.70,3.40,4.50


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
13588,13589,13274,13274,2009/2010,10,2009-10-17 00:00:00,662328,8640,10228,1,...,10.00,1.25,4.50,11.00,1.30,4.50,9.00,1.30,4.33,9.00
13603,13604,13274,13274,2009/2010,11,2009-10-25 00:00:00,662343,8525,10228,4,...,2.25,2.75,3.30,2.30,2.75,3.20,2.35,2.75,3.30,2.30
13620,13621,13274,13274,2009/2010,13,2009-11-08 00:00:00,662360,9908,10228,2,...,3.80,1.80,3.40,4.00,1.80,3.40,4.00,1.80,3.40,4.00
13629,13630,13274,13274,2009/2010,14,2009-11-22 00:00:00,662369,8593,10228,5,...,8.50,1.20,5.50,11.00,1.30,4.75,8.00,1.29,4.50,10.00
13650,13651,13274,13274,2009/2010,16,2009-12-06 00:00:00,662390,9791,10228,3,...,2.75,2.38,3.25,2.62,2.35,3.25,2.70,2.38,3.25,2.70
13668,13669,13274,13274,2009/2010,18,2010-01-20 00:00:00,662408,8674,10228,2,...,3.50,2.10,3.20,3.25,2.10,3.25,3.20,2.20,3.30,2.88
13680,13681,13274,13274,2009/2010,2,2009-08-07 00:00:00,662258,8464,10228,4,...,2.63,2.40,3.30,2.60,2.50,3.30,2.50,2.40,3.30,2.60
13691,13692,13274,13274,2009/2010,20,2010-01-30 00:00:00,662422,10217,10228,2,...,2.20,3.20,3.25,2.10,3.20,3.25,2.10,2.88,3.30,2.20
13699,13700,13274,13274,2009/2010,21,2010-02-02 00:00:00,662430,9839,10228,3,...,2.60,2.38,3.00,2.88,2.50,3.20,2.60,2.38,3.25,2.70
13730,13731,13274,13274,2009/2010,24,2010-02-20 00:00:00,662461,10219,10228,1,...,1.80,4.00,3.40,1.80,3.75,3.30,1.90,3.60,3.30,1.91


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
